In [1]:
import os

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import numpy as np
import torch
import random
import math 
import copy
import random
import argparse
import torch.optim as optim
import torch.nn as nn
import numpy as np
from scipy.optimize import minimize
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import numpy as np

cuda:0


In [2]:
import torch
import torch.nn as nn

# 定义一个简单的神经网络模型
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(hidden_size, output_size)
        self.relu4 = nn.ReLU()

        # 使用Xavier初始化来初始化权重
        nn.init.xavier_normal_(self.fc1.weight)
        nn.init.xavier_normal_(self.fc2.weight)

        # 初始化偏置为零
        nn.init.constant_(self.fc1.bias, 0)
        nn.init.constant_(self.fc2.bias, 0)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.fc4(out)
        out = self.relu4(out)
        return out
        
# 定义输入、隐藏层和输出的维度
input_size = 10
hidden_size = 20
output_size = 10

# 创建一个SimpleNN模型



In [16]:
class customloss(nn.Module):
    def __init__(self, theta):
        super(customloss, self).__init__()
        self.theta = theta
    def forward(self, output_list, y_list):
        theta = torch.tensor(self.theta, dtype= torch.float32).to(device) 
        loss = 0
        index = 0
        for output in output_list:  
            y = torch.tensor(y_list[index]).to(device) 
            v = torch.mm(output, theta.view(-1,1)) 
            prob = torch.exp(v) / (torch.sum(torch.exp(v)))  
            loss += torch.sum(-y * torch.log(prob) - (1-y) * torch.log(1-prob)) 
            index += 1  
        return loss / (len(y_list)/100)

class CustomLoss2(nn.Module):
    def __init__(self):
        super(CustomLoss2, self).__init__()

    def forward(self, output_list, y_list, theta):
        theta = torch.tensor(theta, dtype=torch.float32).to(device)
        loss = 0
        for output, y in zip(output_list, y_list):
            y = torch.tensor(y).to(device) 
            v = torch.matmul(output, theta.view(-1,1))
            prob = torch.exp(v) / torch.sum(torch.exp(v))
            loss += torch.sum(-y * torch.log(prob) - (1-y) * torch.log(1-prob))
        return loss / len(y_list)

In [17]:
CONTEXT_ARRAY = np.load('features1.npy')
theta = np.load('theta1.npy')
PURCHASE_LIST = np.load('purchase1.npy')

In [19]:
feature_list = []
purchase_list = []

model = SimpleNN(input_size, hidden_size, output_size).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=0.01)
for t in range(0,10000):
    context = CONTEXT_ARRAY[t]

    purchase_list.append(PURCHASE_LIST[t])

    feature_list.append(np.array(context))

    if t%10 == 1:
        loss_function = CustomLoss2()
        epochs = 1
        for epoch in range(epochs):
            
            output_list = [model(torch.tensor(a,dtype = torch.float32).to(device)) for a in feature_list]
            loss = loss_function(output_list, purchase_list,theta)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
         
            print('epoch {}, loss {}'.format(epoch, loss.item()))

epoch 0, loss 24.13852882385254
epoch 0, loss 24.131473541259766
epoch 0, loss 24.128826141357422
epoch 0, loss 24.128662109375
epoch 0, loss 24.127685546875
epoch 0, loss 24.127132415771484
epoch 0, loss 24.127050399780273
epoch 0, loss 24.12675666809082
epoch 0, loss 24.126142501831055
epoch 0, loss 24.12552261352539
epoch 0, loss 24.125221252441406
epoch 0, loss 24.12517738342285
epoch 0, loss 24.124704360961914
epoch 0, loss 24.124469757080078
epoch 0, loss 24.124183654785156
epoch 0, loss 24.123916625976562
epoch 0, loss 24.123685836791992
epoch 0, loss 24.12352180480957
epoch 0, loss 24.12343978881836
epoch 0, loss 24.123416900634766
epoch 0, loss 24.1231632232666
epoch 0, loss 24.122879028320312
epoch 0, loss 24.12278938293457
epoch 0, loss 24.122514724731445
epoch 0, loss 24.122350692749023
epoch 0, loss 24.122177124023438
epoch 0, loss 24.121973037719727
epoch 0, loss 24.121782302856445
epoch 0, loss 24.121583938598633
epoch 0, loss 24.12140464782715
epoch 0, loss 24.121248245

KeyboardInterrupt: 

In [20]:
problist  = np.load('prob_list.npy')
def hhh(i):
    ff = model(torch.tensor(CONTEXT_ARRAY[i],dtype = torch.float32).to(device))
    ff = ff.cpu().detach().numpy()
    u = (ff @ theta.T ).reshape(1,-1)
    prob = np.exp(u) / (np.sum(np.exp(u)))
    
    print(problist[i])
    print(prob)

hhh(100)   

FileNotFoundError: [Errno 2] No such file or directory: 'prob_list.npy'